# SIADS Milestone I

- **Cydia Tsang (cydia@umich.edu)**, School of Information, University of Michigan
- **Yao Tong (tonyao@umich.edu)**, School of Information, University of Michigan



In [2]:
import sys, os
import json
import pandas as pd
from boto3.dynamodb.types import TypeDeserializer
sys.path.append(os.getcwd() + "/src")
from src.pj_config.dynamo_db_client import get_dynamo_db_client
from decimal import Decimal

# 1. Data Improt

In [23]:
def convert_dynamodb_response(response):
    deserializer = TypeDeserializer()
    items = response['Items']

    def parse_item(item):
        return {key: deserializer.deserialize(value) for key, value in item.items()}

    items = [parse_item(item) for item in items]
    df = pd.DataFrame(items)
    
    df = df.applymap(lambda x: x if not isinstance(x, dict) else x.get('S', x))

    return df
    
dynamodb = get_dynamo_db_client()

response = dynamodb.scan(
    TableName='tiktok_trending',
    Limit=2  # Limit the result to 2 items
)
json_result = convert_dynamodb_response(response)
print(json_result)

0    {'coverThumb': 'https://p16-sign-va.tiktokcdn....
1    {'coverThumb': 'https://p16-sign-va.tiktokcdn....
Name: musicMeta, dtype: object


# 2. Data Cleaning

# 3. Data Manipulation

# 4. Data Visualisation